In [1]:
import sys
sys.path.append('/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/pytorch/1.13.1-CUDA-11.8.0/lib/python3.9/site-packages')

In [2]:
import sys
print(sys.executable)

/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python


In [3]:
import torch

if torch.cuda.is_available():
    print("CUDA is enabled. GPU is available.")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not enabled. GPU is not available.")


CUDA is enabled. GPU is available.
Device count: 1
Current device: 0
Device name: Tesla V100-PCIE-32GB


In [11]:
!{sys.executable} -m pip install torchvision==0.14.1 --no-deps --no-cache-dir


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.2 MB 3.0 MB/s eta 0:00:01��████    | 21.1 MB 3.0 MB/s eta 0:00:02
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.


In [27]:
import sys
!{sys.executable} -m pip install numpy==1.24.4 --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 17.3 MB 2.6 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs3ee5a9eaf8dec5eb000010b0'

You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.


In [18]:
!{sys.executable} -m pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.6 MB 3.1 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1 requires torch==1.13.1, which is not installed.
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.


In [5]:

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import shutil
import json


# Data Preprocessing (convert labels to YOLOv5 format)

CLASS_MAP = {
    'car': 0,
    'bus': 1,
    'truck': 2,
    'person': 3,
    'bike': 4,
    'motor': 5,
    'traffic light': 6,
    'traffic sign': 7,
    'train': 8,
    'rider': 9
}

def convert_box_yolov5(box2d, w, h):
    x1, y1, x2, y2 = box2d['x1'], box2d['y1'], box2d['x2'], box2d['y2']
    x_center = (x1 + x2) / 2.0 / w
    y_center = (y1 + y2) / 2.0 / h
    bw = (x2 - x1) / w
    bh = (y2 - y1) / h
    return x_center, y_center, bw, bh

def convert_annotations(label_dir, image_dir, output_img_dir, output_label_dir):
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    converted = 0
    for img_name in os.listdir(image_dir):
        if not img_name.endswith('.jpg'):
            continue
        img_path = os.path.join(image_dir, img_name)
        json_path = os.path.join(label_dir, img_name.replace('.jpg', '.json'))
        if not os.path.exists(json_path):
            continue
        with open(json_path, 'r') as f:
            data = json.load(f)
        frame = data.get('frames', [{}])[0]
        objects = frame.get('objects', [])
        lines = []
        image = Image.open(img_path)
        w, h = image.size
        for obj in objects:
            if 'box2d' not in obj:
                continue
            cls = obj['category']
            if cls not in CLASS_MAP:
                continue
            x, y, bw, bh = convert_box_yolov5(obj['box2d'], w, h)
            lines.append(f"{CLASS_MAP[cls]} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}")
        if lines:
            label_file = os.path.join(output_label_dir, img_name.replace('.jpg', '.txt'))
            with open(label_file, 'w') as f:
                f.write('\n'.join(lines))
            shutil.copy(img_path, os.path.join(output_img_dir, img_name))
            converted += 1
    print(f" Converted {converted} images to YOLOv5 format")



def evaluate_model(model, dataloader, device='cuda' if torch.cuda.is_available() else 'cpu', max_batches=3):
    model.eval()
    model.to(device)
    print("\n Running evaluation on a few batches...")
    with torch.no_grad():
        for batch_idx, (imgs, targets) in enumerate(dataloader):
            if batch_idx >= max_batches:
                break
            imgs = imgs.to(device)
            outputs = model(imgs)

            print(f"\nBatch {batch_idx + 1}")
            print(f" - Input shape: {imgs.shape}")
            print(f" - Output shape: {outputs.shape}")

            b, ch, h, w = outputs.shape
            anchors = 3
            num_classes = int(ch / anchors - 5)
            outputs = outputs.view(b, anchors, 5 + num_classes, h, w).permute(0, 1, 3, 4, 2)

            conf = torch.sigmoid(outputs[..., 4])
            class_scores = torch.sigmoid(outputs[..., 5:])

            print(f" - Objectness score (min/max): {conf.min().item():.4f} / {conf.max().item():.4f}")
            print(f" - Class scores (min/max): {class_scores.min().item():.4f} / {class_scores.max().item():.4f}")
    print("\n Evaluation complete.")


/user/hgowdarm/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [6]:


# Model Definition (YOLOv5-like)

def conv_bn(in_channels, out_channels, kernel_size, stride):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=kernel_size // 2, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.1)
    )

class CSPBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CSPBlock, self).__init__()
        hidden = out_channels // 2
        self.conv1 = conv_bn(in_channels, hidden, 1, 1)
        self.conv2 = conv_bn(hidden, hidden, 3, 1)
        self.conv3 = conv_bn(hidden * 2, out_channels, 1, 1)

    def forward(self, x):
        y1 = self.conv2(self.conv1(x))
        y2 = self.conv1(x)
        return self.conv3(torch.cat([y1, y2], dim=1))

class SPPF(nn.Module):
    def __init__(self, in_channels, out_channels, pool_size=5):
        super(SPPF, self).__init__()
        hidden = in_channels // 2
        self.conv1 = conv_bn(in_channels, hidden, 1, 1)
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=1, padding=pool_size // 2)
        self.conv2 = conv_bn(hidden * 4, out_channels, 1, 1)

    def forward(self, x):
        x = self.conv1(x)
        y1 = self.pool(x)
        y2 = self.pool(y1)
        y3 = self.pool(y2)
        return self.conv2(torch.cat([x, y1, y2, y3], 1))

class YOLOv5Custom(nn.Module):
    def __init__(self, num_classes=10):
        super(YOLOv5Custom, self).__init__()
        self.layer1 = conv_bn(3, 32, 3, 1)
        self.layer2 = conv_bn(32, 64, 3, 2)
        self.csp1 = CSPBlock(64, 64)
        self.layer3 = conv_bn(64, 128, 3, 2)
        self.csp2 = CSPBlock(128, 128)
        self.layer4 = conv_bn(128, 256, 3, 2)
        self.csp3 = CSPBlock(256, 256)
        self.sppf = SPPF(256, 256)
        self.head = nn.Sequential(
            conv_bn(256, 128, 3, 1),
            nn.Conv2d(128, 3 * (5 + num_classes), 1, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.csp1(x)
        x = self.layer3(x)
        x = self.csp2(x)
        x = self.layer4(x)
        x = self.csp3(x)
        x = self.sppf(x)
        return self.head(x)


In [7]:

# Loss Function 

class YOLOLoss(nn.Module):
    def __init__(self, lambda_coord=5.0, lambda_noobj=0.5):
        super(YOLOLoss, self).__init__()
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, predictions, targets):
        return torch.mean(predictions ** 2)


# Dataset + Collate

class YOLODataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None, limit=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        if limit:
            self.image_files = self.image_files[:limit]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_file)
        label_path = os.path.join(self.label_dir, img_file.replace('.jpg', '.txt'))

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    boxes.append([cls, x, y, bw, bh])

        return image, torch.tensor(boxes)

def yolo_collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets


# Training Utilities

def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for imgs, targets in dataloader:
        imgs = imgs.to(device)
        preds = model(imgs)
        loss = loss_fn(preds, preds)  # placeholder
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def run_training(model, train_loader, val_loader, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = YOLOLoss()
    for epoch in range(num_epochs):
        loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
        print(f"[Epoch {epoch+1}] Loss: {loss:.4f}")
    torch.save(model.state_dict(), "yolov5_customm_checkpoint.pt")
    print(" Weights saved to yolov5_customm_checkpoint.pt")

# Run Label Conversion Once
if not os.path.exists("custom_yolo_dataset/labels/train"):
    convert_annotations("bdd100k/100k/train", "bdd100k/100k/train",
                        "custom_yolo_dataset/images/train", "custom_yolo_dataset/labels/train")

if not os.path.exists("custom_yolo_dataset/labels/val"):
    convert_annotations("bdd100k/100k/val", "bdd100k/100k/val",
                        "custom_yolo_dataset/images/val", "custom_yolo_dataset/labels/val")
# Dataset Setup

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

train_dataset = YOLODataset("custom_yolo_dataset/images/train", "custom_yolo_dataset/labels/train", transform, limit=20000)
val_full = YOLODataset("custom_yolo_dataset/images/val", "custom_yolo_dataset/labels/val", transform)
val_size = int(0.8 * len(val_full))
test_size = len(val_full) - val_size
val_dataset, test_dataset = random_split(val_full, [val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=yolo_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=yolo_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=yolo_collate_fn)


In [4]:
import numpy as np
print(np.__version__)

1.24.4


In [8]:

# Run Training + Evaluation

model = YOLOv5Custom(num_classes=10)
run_training(model, train_loader, val_loader, num_epochs=5)

# Evaluate model on test data
evaluate_model(model, test_loader)


[Epoch 1] Loss: 0.0004
[Epoch 2] Loss: 0.0000
[Epoch 3] Loss: 0.0000
[Epoch 4] Loss: 0.0000
[Epoch 5] Loss: 0.0000
 Weights saved to yolov5_customm_checkpoint.pt

 Running evaluation on a few batches...

Batch 1
 - Input shape: torch.Size([4, 3, 256, 256])
 - Output shape: torch.Size([4, 45, 32, 32])
 - Objectness score (min/max): 0.4996 / 0.5005
 - Class scores (min/max): 0.4994 / 0.5006

Batch 2
 - Input shape: torch.Size([4, 3, 256, 256])
 - Output shape: torch.Size([4, 45, 32, 32])
 - Objectness score (min/max): 0.4997 / 0.5005
 - Class scores (min/max): 0.4995 / 0.5005

Batch 3
 - Input shape: torch.Size([4, 3, 256, 256])
 - Output shape: torch.Size([4, 45, 32, 32])
 - Objectness score (min/max): 0.4999 / 0.5002
 - Class scores (min/max): 0.4996 / 0.5005

 Evaluation complete.
